In [ ]:
import os
import glob
import numpy as np
import librosa
import scipy.signal


def preprocess_wav_to_spectrogram_matrix(src, dst, print_every=500, fixed_time_dim=169):
    """
    Converts .wav files in a single folder to spectrograms, retrieves targets, 
    and saves them as a matrix and labels as separate arrays.

    Parameters:
    -----------
        src: string
            Path to the directory containing .wav files.
        dst: string
            Path to the directory where the matrix and labels will be saved.
        print_every: int
            Print progress for every 'n' examples.
        fixed_time_dim: int
            Fixed size for the time dimension of spectrograms (default: 169).
    """
    if not os.path.exists(dst):
        os.makedirs(dst)

    spectrograms = []
    targets = []

    # Iterate through all .wav files in the source folder
    for idx, filepath in enumerate(glob.glob(os.path.join(src, "*.wav"))):
        # Extract file name without extension
        filename = os.path.basename(filepath).replace(".wav", "")

        # Extract the target (number 1) from the filename
        target = int(filename.split("_")[0])

        # Load .wav file
        data, fs = librosa.load(filepath, sr=8000)

        # Generate spectrogram using Short-Time Fourier Transform (STFT)
        f, t, Zxx = scipy.signal.stft(data, fs, nperseg=455, noverlap=420, window='hann')
        Zxx = np.abs(Zxx[0:227, :])  # Ensure Zxx shape is consistent

        # Convert to decibels
        Zxx_db = librosa.amplitude_to_db(Zxx, ref=np.max)

        # Adjust spectrogram time dimension to fixed size
        if Zxx_db.shape[1] < fixed_time_dim:
            # Pad with zeros if shorter
            pad_width = fixed_time_dim - Zxx_db.shape[1]
            Zxx_db = np.pad(Zxx_db, ((0, 0), (0, pad_width)), mode='constant')
        elif Zxx_db.shape[1] > fixed_time_dim:
            # Truncate if longer
            Zxx_db = Zxx_db[:, :fixed_time_dim]

        # Add spectrogram and target to lists
        spectrograms.append(Zxx_db)
        targets.append(target)

        # Print only for every 'print_every' example
        if idx % print_every == 0:
            print(f"Processed: {filename}, Target: {target}")

    # Convert lists to NumPy arrays
    spectrograms = np.array(spectrograms)  # Shape: (num_samples, 227, fixed_time_dim)
    targets = np.array(targets)  # Shape: (num_samples,)

    # Save the spectrogram matrix and targets as .npy files
    spectrograms_file = os.path.join(dst, "spectrograms.npy")
    targets_file = os.path.join(dst, "targets.npy")
    np.save(spectrograms_file, spectrograms)
    np.save(targets_file, targets)

    print(f"Saved spectrogram matrix to {spectrograms_file}")
    print(f"Saved targets to {targets_file}")




# Set the source and destination paths
source_path = "C:/Users/mitak/OneDrive - University of Twente/Documents/UTwente/Master Robotics/Module2/Deep Learning/Advesarial-Attack/Model/AudioMNIST-master/AudioMNIST-master/dataAll"
destination_path = "C:/Users/mitak/OneDrive - University of Twente/Documents/UTwente/Master Robotics/Module2/Deep Learning/Advesarial-Attack/dataSpectogram"

# Call the function
preprocess_wav_to_spectrogram_matrix(src=source_path, dst=destination_path, print_every=500)


Processed: 0_01_0, Target: 0
Processed: 0_11_0, Target: 0
Processed: 0_21_0, Target: 0
Processed: 0_31_0, Target: 0
Processed: 0_41_0, Target: 0
Processed: 0_51_0, Target: 0
Processed: 1_01_0, Target: 1
